In [1]:
import src
from src import oic_dwic
from src import ecif
from src import utils
from src import prtcmm_ifp

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (C:\Users\Taniyama\AppData\Roaming\Python\Python38\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
dwic = oic_dwic.InterAtomicContact(
    pathfiles="./test/",
    filename="dwic_fv.csv",
    ligand_format="mol2",
    amino_acid_classes=utils.amino_acid_classes_DWIC,
    cutoff=12.0,
    feature_type="DWIC",
    exp=2,
)

oic = oic_dwic.InterAtomicContact(
    pathfiles="./test/",
    filename="oic_fv.csv",
    ligand_format="mol2",
    amino_acid_classes=utils.amino_acid_classes_OIC,
    cutoff=12.0,
    feature_type="OIC",
    exp=None,
)

ecif = ecif.ECIF(
    pathfiles="./test/", filename="ecif_fv.csv", ligand_format="sdf", cutoff=6.0
)

ifp = prtcmm_ifp.PrtCmmIFP(
    pathfiles=r"./test/", filename="ifp_fv.csv", ligand_format="mol2", parameters=utils.prtcmm_ifp_parameters
)

plec = prtcmm_ifp.PrtCmmIFP(
    pathfiles=r"./test/", filename="plec_fv.csv", ligand_format="mol2", parameters=utils.plec_fp_parameters
)

In [21]:
dwic.generate_features(n_jobs=-1)
oic.generate_features(n_jobs=-1)
ecif.generate_features(n_jobs=-1)
ifp.generate_features(n_jobs=-1)
plec.generate_features(n_jobs=-1)

Time: 0.0:0.0:35.93
Time: 0.0:0.0:17.55
